Si vous vous rappelez comment nous avons terminé le dernier TD, un algorithme simple comme KNN ne donne pas de très bons résultats lorsque la classification devient plus difficile.

Aujourd'hui, nous allons utiliser les réseaux neuronaux artificiels. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import glob
from scipy.io import wavfile
from tqdm  import tqdm
import pickle

dataset_path = "/content/drive/My Drive/IA_MIAGE/google-speech-dataset/"
all_classes = os.listdir(dataset_path)
all_classes.remove('.git')
all_classes.remove('bird') #some outliers in dataset
all_classes.remove('_lab_ressources_')
all_classes.remove('_background_noise_')
chosen_classes = ["three", "tree"]

Comme la semaine dernière, la partie du chargement du dataset a pris beaucoup de temps pour tout le monde. Aujourd'hui, nous allons télécharger les données pour les classes *three* and *tree*.

In [ ]:
% cd /content/drive/'My Drive'/IA_MIAGE/
! wget http://mainline.i3s.unice.fr/CoursIA/X_val_waveform.pkl
! wget http://mainline.i3s.unice.fr/CoursIA/X_train_waveform.pkl
! wget http://mainline.i3s.unice.fr/CoursIA/X_test_waveform.pkl

! wget http://mainline.i3s.unice.fr/CoursIA/y_val.pkl
! wget http://mainline.i3s.unice.fr/CoursIA/y_train.pkl
! wget http://mainline.i3s.unice.fr/CoursIA/y_test.pkl

In [ ]:
X_train_waveform = pickle.load(open("/content/drive/My Drive/IA_MIAGE/X_train_waveform.pkl", "rb"))
X_test_waveform = pickle.load(open("/content/drive/My Drive/IA_MIAGE/X_test_waveform.pkl", "rb"))
X_val_waveform = pickle.load(open("/content/drive/My Drive/IA_MIAGE/X_val_waveform.pkl", "rb"))

y_train =  pickle.load(open("/content/drive/My Drive/IA_MIAGE/y_train.pkl", "rb"))
y_test =  pickle.load(open("/content/drive/My Drive/IA_MIAGE/y_test.pkl", "rb"))
y_val =  pickle.load(open("/content/drive/My Drive/IA_MIAGE/y_val.pkl", "rb"))

Dans la dernière TD, nous avons vu que l'utilisation des fonctions MFCC est une bonne stratégie pour la reconnaissance des paroles. Nous utiliserons donc MFCC pour le TD d'aujourd'hui.

In [ ]:
!pip install sonopy
from sonopy import mfcc_spec

In [ ]:
def get_mfcc_from_signal(signal):

  #extract MFCCs features
  single_mfcc = mfcc_spec(signal, 16000, window_stride=(400, 160), fft_size=512, num_filt=20, num_coeffs=13).T

  #dropping first coefficient
  single_mfcc = single_mfcc[1:,:] #keeping only 12 coefficients

  return single_mfcc

In [ ]:
X_train_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_train_waveform,position=0)]
X_val_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_val_waveform,position=0)]
X_test_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_test_waveform,position=0)]

In [ ]:
X_train_mfcc = np.asarray(X_train_mfcc)
X_val_mfcc = np.asarray(X_val_mfcc)
X_test_mfcc = np.asarray(X_test_mfcc)

X_train_mfcc_flatten = X_train_mfcc.reshape(X_train_mfcc.shape[0], X_train_mfcc.shape[1] * X_train_mfcc.shape[2])
X_val_mfcc_flatten = X_val_mfcc.reshape(X_val_mfcc.shape[0], -1)
X_test_mfcc_flatten = X_test_mfcc.reshape(X_test_mfcc.shape[0], -1)

[Keras](https://keras.io/) est une bibliothèque de réseau neuronal à code source ouvert écrite en Python. Elle peut fonctionner en plus de TensorFlow, Microsoft Cognitive Toolkit, R, Theano ou PlaidML. Conçue pour permettre l'expérimentation rapide de réseaux neuronaux profonds, elle se veut conviviale, modulaire et extensible.

Nous utiliserons *Keras* pour créer nos réseaux de neurones. C'est une bibliothèque très simple, où nous pouvons créer un modèle entier seulement avec quelques lignes de code.

Vous devez d'abord définir votre modèle :
```
model = Sequential()
```

Ensuite, vous pouvez commencer à ajouter des couches à votre modèle :

```
model.add(Dense(2, input_dim=3))
```

La première couche de votre modèle doit avoir le paramètre *input_dim* qui indique la taille de vos données.

Une couche *Dense* dans Keras est une couche contenant des neurones, où chaque neurone de la couche est connecté à toutes les entrées. Pour cette raison, une couche Dense est également appelée couche entièrement connectée (Fully Connected).

Alors, le diagramme suivant, a une entrée de taille 3 et une couche dense de 2 neurones. 


<img src="https://miro.medium.com/max/1890/1*Kg5cA0WNLjDnS3F6gbwFYQ.gif" width="500"/>



The code to replilcate this model would be:

```
model = Sequential()
model.add(Dense(2, input_dim=3))
```

Un réseau neuronal est composé d'entrées, de couches cachées et de sorties. Les couches cachées sont toutes les couches situées entre l'entrée et la sortie. 

Dans l'image suivante, la couche jaune est la couche d'entrée, puis la couche verte sont les couches cachées et la couche rouge est la sortie. 

<img src="https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/74_blog_image_2.png" width="500"/>

Le code en Keras pour cette image serait :


```
model = Sequential()
model.add(Dense(6, input_dim=5))
model.add(Dense(8))
model.add(Dense(2))
```

Keras fournit également une fonction permettant de définir la perte, la métrique et l'optimiseur.

```
model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
```

Nous n'entrerions pas dans les détails de tous les paramètres de la fonction. *sgd* signifie descente de gradient stochastique, *categorical_crossentropy* est une fonction de perte utilisée pour les tâches de classification.






L'image précédente montre deux sorties : Chien et chat. C'est très similaire à notre problème, nous avons l'arbre et trois. 

Si vous vous souvenez des étiquettes de notre ensemble de données, nous avons 0 pour une classe et 1 pour l'autre. Cela signifie que notre classe est unidimensionnelle. 

Habituellement, pour entraîner les réseaux de neurones, nous utilisons une technique appelée *one-hot encoding* pour représenter la classe. Vous créez un vecteur contenant autant de zéros que le nombre de classes, vous attribuez une classe à chaque colonne et vous mettez ensuite 1 dans la colonne correspondante.

Si nous avions trois classes : *Red*, "Blue*, *Greeb*. L'image suivante montre la représentation de ces classes par "one-hot encoding".

<img src="https://www.educative.io/api/edpresso/shot/6284921929728000/image/6316646797934592" width="250"/>

Keras propose une méthode permettant de le faire automatiquement


In [ ]:
import tensorflow as tf
import keras
nb_classes = 2
y_train_cat = tf.keras.utils.to_categorical(y_train, 2)
y_test_cat = tf.keras.utils.to_categorical(y_test, 2)
y_val_cat = tf.keras.utils.to_categorical(y_val, 2)

print("Original class:", y_test[0], ", Class in one-hot encoding:", y_test_cat[0])

Maintenant, créons un modèle ne contenant qu'un seul neurone dans la première couche cachée.

In [ ]:
input_size = X_train_mfcc_flatten.shape[1] ## Size of the mfcc signal

from keras.models import Sequential
from keras.layers import Dense, Activation 

model = Sequential()
model.add(Dense(1, input_dim=input_size, activation='relu')) ## Using only one neuron
model.add(Dense(nb_classes, activation='softmax')) ## Prediction

model.summary() ## This line is used to print the architecture of the model

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

En plus des paramètres que nous avons vus jusqu'à présent, il y en a d'autres liés au training: *Epoch*, *Batch* et *Batch size*.

Au lieu de charger directement tout l'ensemble des données en mémoire, dans les réseaux neuronaux, vous créez des lots de données.Ces lots sont appelés *Batches*.

Un *epoch* est terminé quand le réseau a vu tous les batches du dataset.

Et comme l'optimisation se fait par lots, il est préférable de voir l'ensemble des données plusieurs fois. Pour cette raison, vous entraînez généralement votre réseau pour plusieurs *epochs*.

Créons un réseau de neurones avec un seul neurone dans sa couche cachée.

In [ ]:
print("Training....")
model.fit(X_train_mfcc_flatten, y_train_cat, epochs=1, batch_size=128)



Nous avons choisi d'utiliser une taille de lot de 128 échantillons et d'entraîner pour une seule époque.

Cela signifie que nous avons divisé l'ensemble training en groupes de 128 pistes. Notre ensemble d'entraînement compte 3271 pistes, si nous le divisons en groupes de 128, nous aurons 26 lots.

Nous pouvons également entraîner pour plusieurs époques. En d'autres termes, si vous dites que vous entrainez votre réseau pour 30 époques, cela signifie que votre réseau verra les 3721 échantillons 30 fois.

In [ ]:
print("Training....")
model.fit(X_train_mfcc_flatten, y_train_cat, epochs=30, batch_size=128)

Nous pouvons entraîner le réseau pour autant d'époques que nous le souhaitons. Mais comment savoir quand s'arrêter ?

Comme nous l'avons fait avec KNN, où nous utilisons le ensemble de validation pour choisir la meilleure valeur de k, nous pouvons utiliser le ensemble de validation pour définir quand arrêter l'entreinement.

Keras fournit une méthode qui évalue notre modèle sur l'ensemble de validation chaque fois qu'une *epoch* est terminée et stocke le modèle avec les meilleurs résultats.

```
ourCallback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=20)
```

Cette fonction arrête le training si pendant 20 epochs consécutives il n'y a pas eu une amélioration d'au moins 0,0001



In [ ]:
from keras.callbacks import EarlyStopping

# we define a callback function that will control if the accuracy 
# on the validation set (a part of train set) is not changing more than 10-4 with a patience of 20 iterations
# If the last accuracy value is not the best one, we still keep the last results
ourCallback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=20)

model = Sequential()

model.add(Dense(1, input_dim=input_size, activation='relu')) 

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=100, batch_size=128, callbacks=[ourCallback])

Vous pouvez évaluer votre modèle ne contenant qu'un seul neurone dans la couche cachée

In [ ]:
print("Testing...")
score = model.evaluate(X_test_mfcc_flatten ,y_test_cat)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Vous pouvez essayer d'ajouter d'autres neurones dans la couche cachée.

In [ ]:

nb_neurons =  #A completer: 16, 32, 64, 128, ....

model = Sequential()

model.add(Dense(nb_neurons, input_dim=input_size, activation='relu'))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=100, batch_size=128, callbacks=[ourCallback])

print("Testing...")
score = model.evaluate(X_test_mfcc_flatten ,y_test_cat)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

que se passe-t-il si nous ajoutons une couche cachée supplémentaire ?

In [ ]:
nb_neurons_first_layer = #A completer: 16, 32, 64, 128, ....
nb_neurons_second_layer = #A completer: 16, 32, 64, 128, ....
model = Sequential()

model.add(Dense(nb_neurons_first_layer, input_dim=input_size, activation='relu'))
model.add(Dense(nb_neurons_second_layer, input_dim=input_size, activation='relu'))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=100, batch_size=128, callbacks=[ourCallback])

print("Testing...")
score = model.evaluate(X_test_mfcc_flatten ,y_test_cat)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Vous pouvez analyser les courbes de la perte et la précision. 

Les valeurs de training sont-elles vraiment éloignées des valeurs de validation ?

In [ ]:
import matplotlib.pyplot as plt

nb_neurons_first_layer = #A completer
nb_neurons_second_layer = #A completer
model = Sequential()

model.add(Dense(nb_neurons_first_layer, input_dim=input_size, activation='relu'))
model.add(Dense(nb_neurons_second_layer, input_dim=input_size, activation='relu'))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
history = model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=300, batch_size=128, callbacks=[ourCallback])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Si la perte de training diminue, cela signifie que notre modèle apprend correctement. En revanche, si la perte de validation augmente alors que la perte de training diminue, cela signifie qu'il y a un surajustement (overfittinig).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Overfitting.svg/1200px-Overfitting.svg.png" width="250"/>

La ligne verte représente un modèle trop ajusté (overfitted) et la ligne noire représente un modèle régularisé. Si la ligne verte suit mieux les données de formation, elle est trop dépendante de ces données et il est probable qu'elle présente un taux d'erreur plus élevé sur les nouvelles données invisibles, par rapport à la ligne noire.

Une façon de résoudre le problème du surajustement est d'ajouter les couches Dropout (Décrochage).

<img src="https://miro.medium.com/max/513/1*dEi_IkVB7IpkzZ-6H0Vpsg.png" width="250"/>

Le décrochage, ou abandon, est une technique de régularisation pour réduire le surajustement dans les réseaux de neurones. La technique évite des co-adaptations complexes sur les données de l'échantillon d'entraînement. C'est un moyen très efficace d'exécuter un moyennage du modèle de calcul avec des réseaux de neurones1. Le terme "décrochage" se réfère à une suppression temporaire de neurones (à la fois les neurones cachés et les neurones visibles) dans un réseau de neurones2.

Le réseau neuronal se voit amputé d'une partie de ses neurones pendant la phase d'entrainement (leur valeur est estimée à 0) et ils sont par contre réactivés pour tester le nouveau modèle. [[source: Wikipedia](https://fr.wikipedia.org/wiki/Abandon_(r%C3%A9seaux_neuronaux)]

Dans Keras, on peut ajouter des couches de Dropout après n'importe quelle couche.

```
model.add(Dense(...))
model.add(Dropout(0.5))
```
La valeur à l'intérieur de la parenthèse *Dropout(0.5)* représente la quantité de neurones que nous ignorons de la couche précédente.

Vous pouvez ajouter des couches de décrochage pour voir si la perte de validation est plus proche de la perte de training. N'oubliez pas que toutes les couches ne nécessitent pas d'abandon.




In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Dropout

ourCallback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=100)
nb_neurons_first_layer = #A completer
nb_neurons_second_layer = #A completer
model = Sequential()

model.add(Dense(nb_neurons_first_layer, input_dim=input_size, activation='relu'))
# model.add(Dropout(...)) #A completer
model.add(Dense(nb_neurons_second_layer, input_dim=input_size, activation='relu'))
# model.add(Dropout(...)) #A completer

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
history = model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=500, batch_size=128, callbacks=[ourCallback])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Dropout

ourCallback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=40)
nb_neurons_first_layer = #A completer
nb_neurons_second_layer = #A completer
model = Sequential()

model.add(Dense(nb_neurons_first_layer, input_dim=input_size, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(nb_neurons_second_layer, input_dim=input_size, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
history = model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=500, batch_size=128, callbacks=[ourCallback])

Maintenant que vous connaissez les principales techniques pour entraîner un réseau de neurones, vous pouvez modifier les différents paramètres de votre modèle pour obtenir les meilleurs résultats.

In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Dropout

ourCallback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=100)
nb_neurons_first_layer = #A completer
nb_neurons_second_layer = #A completer
nb_neurons_third_layer = #A completer
...
model = Sequential()

model.add(Dense(nb_neurons_first_layer, input_dim=input_size, activation='relu'))
# model.add(Dropout(...)) #A completer
model.add(Dense(nb_neurons_second_layer, input_dim=input_size, activation='relu'))

# model.add(Dropout(...)) #A completer

model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer='sgd',loss='categorical_crossentropy', metrics=['accuracy'])

print("Training....")
history = model.fit(X_train_mfcc_flatten, y_train_cat, validation_data=(X_val_mfcc_flatten, y_val_cat), epochs=500, batch_size=128, callbacks=[ourCallback])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

print("Testing...")
score = model.evaluate(X_test_mfcc_flatten ,y_test_cat)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Après avoir obtenu les meilleurs résultats, vous pouvez analyser les échantillons qui sont encore mal classés par votre modèle.

In [ ]:
from IPython.display import Audio

predictions = model.predict(X_test_mfcc_flatten)
pred_classes = np.argmax(predictions, axis=1)

mal_classes = np.where(pred_classes != y_test)[0]

idx_to_check = # A Completer
print(mal_classes[idx_to_check])

Audio(X_test_waveform[mal_classes[idx_to_check]],rate=16000)

Vous pouvez maintenant essayer de créer un réseau pour deux autres classes

In [ ]:
chosen_classes = ["...", "..."] # A Completer

X = []
for class_name in chosen_classes:
  class_path = dataset_path + class_name
  for audio_path in glob.glob(class_path + "/*.wav"):
    X.append(audio_path)

y = []
for audio_path in X:
  y.append(audio_path.split('/')[-2])

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size=0.2, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)

print('training samples:',len(X_train))
print('testing samples:',len(X_test))
print('validation samples:',len(X_val))

def normalize(audio_signal):
  audio_signal = np.array(audio_signal) ##A completer avec le code vu le dernier TD
  max_value = max(np.absolute(audio_signal)) ## Get the maximum positive value
  norm_signal = audio_signal / max_value
  return norm_signal

def add_padding(audio_signal, desired_len):
  
  length_signal = len(audio_signal)
  if length_signal < desired_len:
    zeros = np.zeros(desired_len)
    start_signal = random.randint(0, desired_len - length_signal) ## Bonne Question: Pourquoi aléatoire?
    zeros[start_signal: start_signal + length_signal] = audio_signal
    return zeros
    
  return audio_signal

def get_signal(signal_path):
  
  rate, sig = wavfile.read(filename=signal_path)
  
  try:
    sig = sig[:, 0]
  except:
    pass
  
  #normalization
  sig = normalize(sig)

  #standardization of sizes
  sig = add_padding(sig,16000)
  
  return sig

X_train_waveform = [get_signal(path) for path in tqdm(X_train,position=0)]
X_test_waveform = [get_signal(path) for path in tqdm(X_test,position=0)]
X_val_waveform = [get_signal(path) for path in tqdm(X_val,position=0)]

X_train_waveform = np.asarray(X_train_waveform)
X_test_waveform = np.asarray(X_test_waveform)
X_val_waveform = np.asarray(X_val_waveform)

